
# 📘 RAG over a Manually Uploaded Text File (Chroma in Working Directory)

This notebook lets you:
1. **Upload a `.txt` file manually** (Colab or local Jupyter).
2. **Chunk & embed** the text with `sentence-transformers`.
3. **Store** vectors in a **Chroma** DB saved to `./chroma` (working directory).
4. **Ask questions**: a simple **RAG** flow retrieves top snippets and answers via:
   - **OpenAI** (if `OPENAI_API_KEY` is set), or
   - **Local Transformers** fallback (`google/flan-t5-base`) if no OpenAI key.
   
> Works on **Colab** and **Windows/macOS** Jupyter. No `.env`/dotenv needed.


## 0) Install dependencies

In [1]:

import sys
print(sys.version)

# Core libs
!pip -q install langchain langchain-community chromadb sentence-transformers

# For optional local LLM fallback
!pip -q install transformers accelerate


3.10.19 (main, Oct 21 2025, 16:37:10) [Clang 20.1.8 ]


## 1) Configuration

In [2]:

import os
from pathlib import Path

# --- Path where Chroma (vector DB) will be persisted ---
CHROMA_DIR = "./chroma"   # working directory
COLLECTION = "uploaded_text"

# --- Optional: OpenAI ---
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "").strip()
USE_OPENAI = bool(OPENAI_API_KEY)
if USE_OPENAI:
    print("✅ Using OpenAI for generation.")
else:
    print("ℹ️ OPENAI_API_KEY not set — will use local Transformers fallback.")

Path(CHROMA_DIR).mkdir(parents=True, exist_ok=True)
print("CHROMA_DIR =", Path(CHROMA_DIR).resolve())
print("COLLECTION  =", COLLECTION)


✅ Using OpenAI for generation.
CHROMA_DIR = /Volumes/Untitled/Lessons_By_Week/Youtube_Project/Working_Pipelines/chroma
COLLECTION  = uploaded_text



## 2) Upload your `.txt` file

Choose **one** method below:

- **A. Google Colab uploader** – pick a file from your computer.
- **B. Local path** – type/paste an existing path on your machine.
- **C. Jupyter widget** – drag & drop in classic Jupyter (optional).


In [ ]:

# A) Google Colab uploader
# If you're in Colab, run this cell and select a .txt file.
# Otherwise, skip to method B or C.

uploaded_path = None
try:
    from google.colab import files  # type: ignore
    up = files.upload()  # opens file chooser
    if up:
        fname = list(up.keys())[0]
        uploaded_path = fname
        print("Uploaded:", uploaded_path)
except Exception as e:
    # Not in Colab, ignore
    pass


In [3]:
# B) Local path (works anywhere)
manual_path = "/Volumes/Untitled/Lessons_By_Week/Youtube_Project/QA_YOUTUBE_NOTEBOOK/Building_Codes/RAG_TEXT.txt"

if manual_path:
    uploaded_path = manual_path

print("Selected path:", uploaded_path)



Selected path: /Volumes/Untitled/Lessons_By_Week/Youtube_Project/QA_YOUTUBE_NOTEBOOK/Building_Codes/RAG_TEXT.txt


In [ ]:

# C) Jupyter file upload widget (optional; classic Jupyter/Lab)
uploaded_via_widget = True
try:
    import io, ipywidgets as widgets
    from IPython.display import display

    fu = widgets.FileUpload(accept='.txt', multiple=False)
    display(fu)

    def _capture(change):
        global uploaded_via_widget, uploaded_path
        if len(fu.value):
            meta = list(fu.value.values())[0]
            content = meta['content']
            name = meta['metadata']['name']
            with open(name, 'wb') as f:
                f.write(content)
            uploaded_via_widget = name
            uploaded_path = name
            print("Widget uploaded:", uploaded_path)

    fu.observe(_capture, 'value')
except Exception as e:
    # ipywidgets not installed or not supported; ignore
    pass


## 3) Read & clean the text

In [4]:

from pathlib import Path

assert uploaded_path, "No file selected. Use one of the upload methods above."

p = Path(uploaded_path)
assert p.exists(), f"File not found: {p}"

text = p.read_text(encoding="utf-8", errors="ignore")
print(f"Loaded {len(text):,} characters from:", p.resolve())

# Light cleanup
text = text.replace("\r\n", "\n").replace("\r", "\n")
text = "\n".join([line.strip() for line in text.split("\n")])
print("First 500 chars:\n", text[:500])


Loaded 110,700 characters from: /Volumes/Untitled/Lessons_By_Week/Youtube_Project/QA_YOUTUBE_NOTEBOOK/Building_Codes/RAG_TEXT.txt
First 500 chars:
 ===== [1] Delivery – Netflix | Partner Help Center :: https://partnerhelp.netflixstudios.com/hc/en-us/categories/1500000000761-Delivery =====
html
Delivery – Netflix | Partner Help Center
Return to Help Center
Partner Help Center
Sign in
Menu
Delivery
Netflix | Partner Help Center
Delivery
Delivery Specifications
+ Branded Delivery Specifications
+ Non-Branded Delivery Specifications
Near Field 2.0 Stereo Delivery Specifications and Guidelines for Titles Mixed Natively in 2.0
Non Graded Archival


## 4) Chunk → Embed → Persist to Chroma

In [5]:
# --- Chunk → Embed → Persist to Chroma (version-safe) ---

from pathlib import Path
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

# 1) Chunk the text
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=160,
    separators=["\n\n", "\n", ". ", " ", ""],
)
docs = [Document(page_content=c, metadata={"source": str(p.name)}) 
        for c in splitter.split_text(text)]
print(f"Chunks created: {len(docs)}")

# 2) Embedding function (no manual embedding arrays)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    encode_kwargs={"normalize_embeddings": True},
)

# 3) Create (or re-open) the Chroma collection in the working dir
#    NOTE: We pass `embedding_function=embeddings` so vectorstore handles embeddings internally.
vs = Chroma(
    collection_name=COLLECTION,
    persist_directory=CHROMA_DIR,
    embedding_function=embeddings,
)

# 4) (Optional) Clear previous contents of this collection
#    Easiest cross-version reset: use a fresh collection name, or delete the folder manually.
#    If you really want programmatic clearing, uncomment:
# try:
#     # Recreate an empty collection (requires chromadb>=0.4 with delete support)
#     from chromadb import PersistentClient
#     client = PersistentClient(path=CHROMA_DIR)
#     try:
#         client.delete_collection(COLLECTION)
#     except Exception:
#         pass
#     vs = Chroma(
#         collection_name=COLLECTION,
#         persist_directory=CHROMA_DIR,
#         embedding_function=embeddings,
#     )
# except Exception:
#     pass

# 5) Add docs (embeddings computed under the hood)
vs.add_documents(docs)

# 6) Persist if available (older langchain-Chroma exposes .persist(); newer chromadb persists automatically)
try:
    vs.persist()  # no-op on some versions; safe to call if present
except Exception:
    pass

print("✅ Stored in Chroma at:", Path(CHROMA_DIR).resolve())


/opt/anaconda3/envs/Rag_Env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Chunks created: 183


/var/folders/59/_rmmbn7d6kx3y2yg0wqg8ghc0000gn/T/ipykernel_7246/3970925276.py:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/var/folders/59/_rmmbn7d6kx3y2yg0wqg8ghc0000gn/T/ipykernel_7246/3970925276.py:27: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vs = Chroma(


✅ Stored in Chroma at: /Volumes/Untitled/Lessons_By_Week/Youtube_Project/Working_Pipelines/chroma


/var/folders/59/_rmmbn7d6kx3y2yg0wqg8ghc0000gn/T/ipykernel_7246/3970925276.py:57: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vs.persist()  # no-op on some versions; safe to call if present


## 5) Build retriever & choose generator (OpenAI or local Transformers)

In [7]:

from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Reopen collection (simulates separate session)
vs = Chroma(collection_name=COLLECTION, persist_directory=CHROMA_DIR)

retriever = vs.as_retriever(search_kwargs={"k": 5})

def format_docs(docs):
    out = []
    for i, d in enumerate(docs):
        src = d.metadata.get("source", "")
        out.append(f"[{i}] {d.page_content}\n(source: {src})")
    return "\n\n".join(out)

SYSTEM_PROMPT = (
    "You are a helpful assistant. Answer the question **only** from the provided context. "
    "If the answer isn't present, say: 'KAY SAYS I don't see that in the file.'"
     "give full context to make the user understand '"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "{system}"),
    ("human", "Question: {question}\n\nContext:\n{context}\n\nAnswer succinctly:"),
])

# --- Choose generator ---
generator = None
if USE_OPENAI:
    from langchain_openai import ChatOpenAI
    generator = ChatOpenAI(model="gpt-4o-mini", temperature=0)
else:
    # Local Transformers text2text generation via HF pipeline
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

    model_id = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
    hf_pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

    class HFText2TextLLM:
        def __call__(self, prompt_text: str) -> str:
            out = hf_pipe(prompt_text, max_new_tokens=256, truncation=True)
            return out[0]["generated_text"]

    # Wrap prompt execution manually when using local model
    generator = HFText2TextLLM()

# Build chain (LangChain-style if OpenAI; manual if local HF)
if USE_OPENAI:
    chain = (
        RunnableParallel({
            "context": (retriever | format_docs),
            "question": RunnablePassthrough(),
            "system": (lambda _: SYSTEM_PROMPT),
        })
        | prompt
        | generator
        | StrOutputParser()
    )
    print("✅ RAG chain (OpenAI) is ready.")
else:
    # We'll emulate the same behavior for the local model in a function
    def answer_local(question: str) -> str:
        ctx = format_docs(retriever.get_relevant_documents(question))
        full_prompt = (
            f"{SYSTEM_PROMPT}\n\n"
            f"Question: {question}\n\n"
            f"Context:\n{ctx}\n\n"
            f"Answer succinctly:"
        )
        return generator(full_prompt)

    chain = answer_local
    print("✅ RAG function (Local Transformers) is ready.")


✅ RAG chain (OpenAI) is ready.


## 6) Ask questions

In [8]:

def ask(question: str):
    if not question.strip():
        return "Please enter a non-empty question."
    if callable(chain) and not hasattr(chain, "invoke"):
        # Local HF function path
        return chain(question)
    # OpenAI path via LangChain
    return chain.invoke(question)

# Try it:
print(ask("Give me a 1–2 sentence summary of the file."))


The file outlines guidelines for creating and delivering subtitles, forced narratives, and alternate language audio descriptions for content, ensuring that translations are accurate and adhere to specific formatting and technical requirements. It emphasizes the importance of maintaining the integrity of the original language while providing clear translations for non-native speakers.


## 7) (Optional) Interactive Q&A loop

In [9]:

# Run this cell to chat in the console.
# Stop with Ctrl+C (or by interrupting the kernel).
try:
    while True:
        q = input("\nAsk a question (or press Enter to exit): ").strip()
        if not q:
            break
        print("\n--- Answer ---\n", ask(q))
except KeyboardInterrupt:
    pass



--- Answer ---
 The text provides guidelines for creating and delivering textless versions of IMF (Interoperable Master Format) packages for Netflix. It outlines the process for handling forced narrative subtitles, which are necessary for conveying important dialogue or on-screen text without burned-in subtitles. The document emphasizes the need for a list of these forced narratives to be submitted using a specific template and details best practices for delivery.

--- Answer ---
 The file contains guidelines and requirements for audio and video asset delivery, specifically focusing on Composition Playlist (CPL) XML files for different audio formats (Dolby Atmos, 5.1 surround, and 2.0 stereo), file naming conventions, and language codes for subtitles and audio descriptions. It outlines the necessary components of an IMF package, acceptable formats for alternate languages, and the importance of avoiding slurs in translations. Additionally, it includes a list of language codes for vario


## Notes, Tips, and Tweaks

- **Chunk sizes**: For dense documents, try `chunk_size=1000–1500`, `chunk_overlap=150–250`.
- **Retriever depth**: Increase `k` to 8–10 for broader context.
- **Chroma persistence**: Data is saved in `./chroma`. You can delete that folder to reset.
- **Multiple files**: Re-run the *Chunk → Embed → Persist* cell with another file; use different `COLLECTION` names to keep them separate.
- **Local model**: `google/flan-t5-base` is light but not perfect. For better local quality, swap it for a bigger model (e.g., `flan-t5-large`) if your machine/Colab GPU can handle it.
- **OpenAI path**: If you set `OPENAI_API_KEY`, the notebook uses `gpt-4o-mini` for higher-quality answers.
